In [2]:
import os

from qiskit import BasicAer, execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit

from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI

QI_URL = os.getenv('42ca57206e553107fd2180b65ab21c3764c70e5a', 'https://api.quantum-inspire.com/')


authentication = get_authentication()
QI.set_authentication(authentication, QI_URL)
qi_backend = QI.get_backend('QX single-node simulator')

In [3]:
import random
from random import getrandbits

In [27]:
# Creating Alice's Bits and Bases
global qkd
def alice(length):
    alice_bits = []
    alice_bases = []
    global Length
    Length = length
    
    #BIT GENERATION
    for i in range(length):
        alice_bits.append(getrandbits(1))
        
        # 0 means encode in the Z basis and 1 means encode in the X,Y basis
        alice_bases.append(getrandbits(1))
    print(len(alice_bases))
        

    encoded_qubits = [] #an array of qc circuits with 1 qubit each
    for i in range(length):
        # create a brand new quantum circuit called qc
        qc = QuantumCircuit(1,1)

        if alice_bases[i] == 0:
            # 0 Means we are encoding in the z basis
            if alice_bits[i] == 0:
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif alice_bits[i] == 1:
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif alice_bases[i] == 1:
            # 1 Means we are encoding in the x basis
            if alice_bits[i] == 0:
                # We apply an H gate to generate |+>
                qc.h(0)
            elif alice_bits[i] == 1:
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
        global alice_bases_xz
        alice_bases_xz = []
        for i in range(length):
            if(alice_bases[i] == 0):
                alice_bases_xz.append("Z")
            else:
                alice_bases_xz.append("X")
    return encoded_qubits

def key(length, bob_bitstring, measure_bases):
    #Converting Alice Bases from 0,1 to X, Z 
    c_KEY = []
    for x in range(length):
        if (alice_bases_xz[x] == measure_bases[x]):
            c_KEY.append(bob_bitstring[x])
            
    return c_KEY

def bowser_key(c_KEY): 
    x = []
    if len(c_KEY) >=3:
        #Replace 1 of Key values
        for i in range(len(c_KEY)):
            x.append(i)
        y = random.randint(0,len(c_KEY)-1)
        corrupted_key = c_KEY
        if c_KEY[y] == '0':
            corrupted_key[y] = '1'
        else:
            corrupted_key[y] = '0'
        x.pop(y)
        
        #Replace second key value
        y = random.randint(0,len(c_KEY)-1)
        corrupted_key = c_KEY
        if c_KEY[y] == '0':
            corrupted_key[y] = '1'
        else:
            corrupted_key[y] = '0'
        
        return corrupted_key
    
    elif len(c_KEY)>1:
        y = getrandbits(1)
        corrupted_key = c_KEY
        if c_KEY[y] == '0':
            corrupted_key[y] = '1'
        else:
            corrupted_key[y] = '0'
        
        return corrupted_key
    
    elif len(c_KEY)==1:
        corrupted_key = c_KEY
        if c_KEY[0] == '0':
            corrupted_key[0] = '1'
        else:
            corrupted_key[0] = '0'
        
        return corrupted_key
        
    
    else:
        while len(c_KEY) == 0:
            enc_q = alice(length)
            measure_bases = select_measurement()
            x = b_measure(measure_bases, enc_q, qi_backend)
            c_KEY = key(length, x, measure_bases)
        

In [25]:
#note: when running function, code must be "measure_bases = ''.join(select_measurement())"
#user inputted bases
global measure_bases
def select_measurement():
    print("Choose 5 bases to measure in using 'Z' and 'X'.\n Z represents the Z basis and X the X basis")
    #changes string input to an array
    measure_bases = [c for c in input("Input your bases: ")] 
  #if there are not 5 bases
    while len(measure_bases) != 5:
        measure_bases = [c for c in input("you did not input 5 bases. Try again: ")]
        #if bases are not Zs and Xs
        for x in range(length):
            while measure_bases[x] != 'Z' and measure_bases[x] != 'X':
                measure_bases = [c for c in input("you did not input X or Z. Try again: ")]
    return measure_bases


def b_measure(measure_bases, encoded_qubits, backend):
    # Perform measurement on the qubits send by Alice
    # selected_measurements: 
    # encoded_qubits: list of QuantumCircuits received from Alice
    # backend: IBMQ backend, either simulation or hardware
    
    # Stores the results of Bob's measurements
    bob_bitstring = ''
    #measure_bases = ''.join(measure_bases)
    bob_mlist = []
    for i in range(Length): #global Length variable = input number of qubits from user
        qc = encoded_qubits[i]
        
        if measure_bases[i] == "Z":
            #change string value to measurement basis
            #qc.measure(0,0)
            pass

        elif measure_bases[i] == "X":
            # change X to measurement basis
            qc.h(0)
            #qc.measure(0,0)
        job = execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bitstring += measured_bit 
            
        
    return bob_bitstring

def QKD(length):
    bowser_involved = getrandbits(1) #random for whether or not Bowser has access to the quantum channel
    
    enc_q = alice(length)
    measure_bases = select_measurement()
    x = b_measure(measure_bases, enc_q, qi_backend)
    c_KEY = key(length, x, measure_bases)
        
    if bowser_involved == 0:
        corrupted_key = bowser_key(c_KEY)
        channel_secure = True
        return corrupted_key, c_KEY, channel_secure
    
    else:
        channel_secure = False
        corrupted_key = []
        for i in range(len(c_KEY)):
            corrupted_key.append(getrandbits(1))
        return c_KEY, corrupted_key, channel_secure
        
    
def compromised_guess(c_KEY, corrupted_key, channel_secure):
    #c_KEY = the final key AFTER checking which gates don't match up and deleting the bits that don't match
    #corrupted_key = the correct key that has been modified
    
    print("Here are Alice's bits: " + str(c_KEY))
    print("Here are your bits: " + str(corrupted_key))
    guess = input("Do you believe that the quantum channel has been compromised? Answer 'YES' or 'NO'")
   
    if (guess == 'YES') or (guess == "yes") or (guess == "Yes"):
        while (guess == 'YES'):
            bowser_involved = getrandbits(1) #random for whether or not Bowser has access to the quantum channel
            
            print("Don't forget, it could also just be some Quantum Noise!")
            enc_q = alice(length)
            measure_bases = select_measurement()
            x = b_measure(measure_bases, enc_q, qi_backend)
            c_KEY = key(length, x, measure_bases)

            if bowser_involved == 0:
                corrupted_key = bowser_key(c_KEY)
                channel_secure = True
                return corrupted_key, c_KEY, channel_secure

            else:
                channel_secure = False
                corrupted_key = []
                for i in range(len(c_KEY)):
                    corrupted_key.append(getrandbits(1))
            print("Here are Alice's bits: " + str(c_KEY))
            print("Here are your bits: " + str(corrupted_key))
            guess = input("Do you believe that the quantum channel has been compromised? Answer 'YES' or 'NO'")
        
        
    if (guess == 'NO') or (guess == "No") or (guess == "no"):
        pass
        
    if channel_secure == True:
        qkd = True
    
    else:
        qkd = False

    return qkd
        
def level_1(length):
    c_KEY, corrupted_key, channel_secure = QKD(length)
    qkd = compromised_guess(c_KEY, corrupted_key, channel_secure)
    
    #TRIVIA = = = (GET CODE!)

def level_2(length):
    c_KEY, corrupted_key, channel_secure = QKD(length)
    qkd = compromised_guess(c_KEY, corrupted_key, channel_secure)
    
    #(GET CODE!)
    
def level_3(length):
    c_KEY, corrupted_key, channel_secure = QKD(length)
    qkd = compromised_guess(c_KEY, corrupted_key, channel_secure)
    
    #(GET CODE!)
    
    #INSERT FINAL BIG CHALLENGE TO DEFEAT BOWSER!

In [26]:
#GAME_STARTS!!!
game_starts = input("Do you want to play QuantuMario? Answer 'YES' or 'NO' ")
global win
if (game_starts == "YES") or (game_starts == "yes") or (game_starts == "Yes"):
    #insert on_start() aka Exposition (GET CODE!)
    #Second Exposition = instructions (GET CODE!)
    length = 5
    level_1(length) #takes care of trivia as well
    print("Congratulations on passing Level 1! Now onto level 2, lets find out what mischief Bowser has been up to")
    level_2(length)
    print("Congratulations on passing Level 2! Now onto level 3, lets find out what mischief Bowser has been up to")
    level_3(length)
    print("Congratulations on passing Level 3! Now onto the Final Quantum Challenge, looks like Bowser learned the secrets of quantum mischief!")
    if win == True:
        print("CONGRATULATIONS BLAH BLAH BLAH")
    
    else:
        print("Sad to see another great hero lose. . .Bowser lives on. . .")

else:
    print("Sorry to hear that!")
    pass
    

Do you want to play QuantuMario? Answer 'YES' or 'NO'  YES


5
Choose 5 bases to measure in using 'Z' and 'X'.
 Z represents the Z basis and X the X basis


Input your bases:  XXXXX


Here are Alice's bits: ['0', '1', '1']
Here are your bits: [1, 0, 1]


Do you believe that the quantum channel has been compromised? Answer 'YES' or 'NO' YES


...

Input your bases:  ZZZZX


5
Choose 5 bases to measure in using 'Z' and 'X'.
 Z represents the Z basis and X the X basis


Input your bases:  XXZXZ


5
Choose 5 bases to measure in using 'Z' and 'X'.
 Z represents the Z basis and X the X basis


KeyboardInterrupt: Interrupted by user

In [ ]:
###### UPDATED JAN 30 12:23 AM